In [1]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np

from os import path

from plotly import express as px

sys.path.append("../")

from ventilatormodels import SimpleRegressor, LSTMRegressor, VarLSTMRegressor
from tqdm import tqdm

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
if not path.isdir("kaggle-data"):
    print("Downloading data...")
    os.mkdir("kaggle-data")
    
    cmds = [
        "cd",
        "kaggle-data",
        "&&",
        "kaggle",
        "competitions",
        "download",
        "-c",
        "ventilator-pressure-prediction",
        "&&",
        "unzip ventilator-pressure-prediction.zip",
    ]
    
    cmd_text = " ".join(cmds)
    
    res = subprocess.run(cmd_text, shell=True, stdout=subprocess.PIPE)
    print("Downloaded and extracted all data!")

In [17]:
training_data = pd.read_csv("kaggle-data/train.csv")
testing_data = pd.read_csv("kaggle-data/test.csv")
training_data

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [18]:
fig = px.line(training_data.head(80*10), x="time_step", y="pressure", color="breath_id")
fig.show()

In [19]:
feature_cols = ["R", "C", "time_step", "u_in", "u_out", "pressure"]

means = training_data[feature_cols].mean()
stdeevs = means = training_data[feature_cols].std()

training_data[feature_cols] = (training_data[feature_cols] - means) / stdeevs
training_data

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,0.020643,1.915059,-1.000000,-0.993797,-1.000000,-0.280184
1,2,1,0.020643,1.915059,-0.956066,0.368325,-1.000000,-0.271515
2,3,1,0.020643,1.915059,-0.911859,0.675458,-1.000000,-0.028786
3,4,1,0.020643,1.915059,-0.867434,0.697754,-1.000000,0.448003
4,5,1,0.020643,1.915059,-0.822768,0.887340,-1.000000,0.508685
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,1.551607,-0.416988,2.269812,-0.889114,1.060686,-0.522913
6035996,6035997,125749,1.551607,-0.416988,2.313361,-0.889205,1.060686,-0.522913
6035997,6035998,125749,1.551607,-0.416988,2.357026,-0.883959,1.060686,-0.531582
6035998,6035999,125749,1.551607,-0.416988,2.400548,-0.905270,1.060686,-0.496907


In [6]:
TS_cols = ["R", "C", "time_step", "u_in", "u_out"]
gp = training_data.groupby(["breath_id", "id"]).mean()
breath_values = list(set(training_data["breath_id"]))
X = np.array([gp.loc[i, TS_cols] for i in breath_values])
X.shape

(75450, 80, 5)

In [7]:
Y = np.array([gp.loc[i, "pressure"] for i in breath_values])
Y.shape

(75450, 80)

In [8]:
X_train = X[:70000, :, :]
Y_train = Y[:70000, :]

X_test = X[70000:, :, :]
Y_test = Y[70000:, :]

In [ ]:
lstm = LSTMRegressor()
es = EarlyStopping(monitor="val_loss", patience=25, verbose=1, restore_best_weights=True)
history = lstm.model.fit(X_train, Y_train, epochs=100, verbose=1, validation_data=(X_test, Y_test), callbacks=[es])

Epoch 1/100
2188/2188 [==============================] - 90s 41ms/step - loss: 0.2419 - val_loss: 0.1790
Epoch 2/100
2188/2188 [==============================] - 87s 40ms/step - loss: 0.1477 - val_loss: 0.1327
Epoch 3/100
2188/2188 [==============================] - 87s 40ms/step - loss: 0.1246 - val_loss: 0.1178
Epoch 4/100
2188/2188 [==============================] - 87s 40ms/step - loss: 0.1149 - val_loss: 0.1120
Epoch 5/100
2188/2188 [==============================] - 86s 40ms/step - loss: 0.1089 - val_loss: 0.1004
Epoch 6/100
2188/2188 [==============================] - 88s 40ms/step - loss: 0.1012 - val_loss: 0.0963
Epoch 7/100
2188/2188 [==============================] - 87s 40ms/step - loss: 0.0975 - val_loss: 0.0919
Epoch 8/100
2188/2188 [==============================] - 87s 40ms/step - loss: 0.0937 - val_loss: 0.0912
Epoch 9/100
2188/2188 [==============================] - 85s 39ms/step - loss: 0.0905 - val_loss: 0.0890
Epoch 10/100
2188/2188 [==============================]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-45833d9150f6>", line 3, in <module>
    history = lstm.model.fit(X_train, Y_train, epochs=100, verbose=1, validation_data=(X_test, Y_test), callbacks=[es])
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\keras\engine\training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\tensorflow\python\eager\def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-pack

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-45833d9150f6>", line 3, in <module>
    history = lstm.model.fit(X_train, Y_train, epochs=100, verbose=1, validation_data=(X_test, Y_test), callbacks=[es])
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\keras\engine\training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-packages\tensorflow\python\eager\def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\trevo\AppData\Local\pypoetry\Cache\virtualenvs\kaggle-ventilator-model-bAWO32nX-py3.9\lib\site-pack

In [20]:
testing_data[TS_cols] = (testing_data[TS_cols] - means[:-1]) / stdeevs[:-1]

gp = testing_data.groupby(["breath_id", "id"]).mean()
breath_values = list(set(testing_data["breath_id"]))
X_test = np.array([gp.loc[i, TS_cols] for i in breath_values])
Y_test = lstm.model.predict(X_test)


array([[[-0.22298451],
        [-0.19799365],
        [-0.11991321],
        ...,
        [-0.20714988],
        [-0.20734029],
        [-0.20747407]],

       [[-0.25048876],
        [-0.18331878],
        [-0.10085842],
        ...,
        [-0.23581366],
        [-0.23568559],
        [-0.23554972]],

       [[-0.25727865],
        [-0.14553152],
        [ 0.23910952],
        ...,
        [-0.20327727],
        [-0.2029592 ],
        [-0.20265397]],

       ...,

       [[-0.30718714],
        [-0.18046738],
        [ 0.8399955 ],
        ...,
        [-0.18668996],
        [-0.18908383],
        [-0.1912377 ]],

       [[-0.24630453],
        [-0.16905443],
        [ 0.18906498],
        ...,
        [-0.2000338 ],
        [-0.19980894],
        [-0.19959025]],

       [[-0.22860354],
        [-0.20303991],
        [ 0.11712321],
        ...,
        [-0.21758074],
        [-0.21753724],
        [-0.21749242]]], dtype=float32)

In [25]:
Y_predictions = (Y_test * stdeevs[-1] + means[-1]).reshape((Y_test.shape[0]*Y_test.shape[1],1))
Y_predictions.shape

(4024000, 1)

In [29]:
final_df = pd.DataFrame()
final_df["id"] = testing_data["id"]
final_df["pressure"] = Y_predictions
final_df.to_csv("SampleModelOutput.csv", index=False)